In [ ]:
!wget -O 2.protein_lengths.tsv https://drive.google.com/uc?id=1cWYjRIGdLZG39j5FmZKgbYaOeWFTLHrD

--2024-03-15 20:41:48--  https://drive.google.com/uc?id=1cWYjRIGdLZG39j5FmZKgbYaOeWFTLHrD
Resolving drive.google.com (drive.google.com)... 74.125.199.139, 74.125.199.100, 74.125.199.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.199.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1cWYjRIGdLZG39j5FmZKgbYaOeWFTLHrD [following]
--2024-03-15 20:41:48--  https://drive.usercontent.google.com/download?id=1cWYjRIGdLZG39j5FmZKgbYaOeWFTLHrD
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.199.132, 2607:f8b0:400e:c02::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.199.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29258442 (28M) [application/octet-stream]
Saving to: ‘2.protein_lengths.tsv’

2.protein_lengths.t 100%[===================>]  27.90M  56.4MB/s    in 0.5s    

2024-03-15 20:41:52 (56.

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [ ]:
protein_lengths = pd.read_csv('2.protein_lengths.tsv', sep='\t')
human_protein_lengths = protein_lengths[protein_lengths["Common name"] == "Human"].copy()
human_protein_lengths["LogLength"] = np.log10(human_protein_lengths["Protein length"])

In [25]:
from scipy.stats import bootstrap

def zeta_function(x):
  return 10**np.mean(x)

Nboot = 5000
res_zeta = bootstrap((human_protein_lengths['LogLength'],), zeta_function, n_resamples=Nboot, vectorized=False, method='percentile', confidence_level=0.95)
res_mu = bootstrap((human_protein_lengths['Protein length'],), np.mean, n_resamples=Nboot, vectorized=False, method='percentile', confidence_level=0.95)

print(f"mu CI (95%): {res_mu.confidence_interval}")
print(f"zeta CI (95%): {res_zeta.confidence_interval}")


mu CI (95%): ConfidenceInterval(low=688.7742525313342, high=696.649406724281)
zeta CI (95%): ConfidenceInterval(low=512.661567283583, high=516.7971454273957)


In [26]:
hist_zeta = go.Histogram(x=res_zeta.bootstrap_distribution, name='Zeta', marker=dict(color='blue'))

hist_mu = go.Histogram(x=res_mu.bootstrap_distribution, name='Mu', marker=dict(color='red'))

fig = go.Figure(data=[hist_zeta, hist_mu])
fig.update_layout(
    title_text='Bootstrap for Zeta and Mu',
    bargap=0.2,
    barmode='overlay'
)

fig.show()